# 1️⃣ **Описание шаблона для решения задачи.**

**Задача**: обучить несколько бустингов на 3-х фолдах, выбрать лучшие, усреднить предсказания.

**Модели, которые будем обучать:**
- `CatBoostRegressor`
- `LightGBMRegressor (goss)`
- `XGBoostRegressor (dart)`


✅ Будут выполнены:
- все дополнительные условия
- возможности фреймворков (загрузка датасетов с помощью соответствующих классов, правильная подготовка категориальных признаков, early_stopping, многопоточность)
- подбор гиперпараметров для каждой модели

👀 При желании, рекомендуется проделать следующее:
- Провести EDA (Exploratory Data Analysis) и сделать выводы на основе графики
- Провести Feature Selection
- Провести Object Selection
- Использовать scheduler или custom callbacks
- Обучить дополнительные модели


❗️❗️❗️ **P.S.**
- Данный ноутбук - далеко не единственное верное решение, воспринимайте его как помощник для вашего собственного решения или чтобы побороть страх белого листа :)

- При полном заполнении ноутбука можно получить максимум 9 баллов из 10, так как из дополнительных баллов - только балл за подбор гиперпараметров.

- При любых найденных ошибках/опечатках/непонятных моментов в коде, пишите в [чат курса](https://stepik.org/lesson/681941/step/6?unit=680724)

# 2️⃣ **Подключение необходимых библиотек и загрузка данных.**

In [ ]:
!pip install catboost lightgbm xgboost -q

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostRegressor, Pool

import lightgbm as lgb
from lightgbm import Dataset, LGBMRegressor

import xgboost as xgb
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_test.csv')

In [ ]:
RANDOM_STATE = ... # Ваше любимое число :)

In [ ]:
results = [] # Здесь будем хранить информацию по каждой модели

# 3️⃣ **Определим вспомогательные функции.**

In [ ]:
def train_model(algorithm,
                X,
                y,
                early_stopping_rounds,
                init_params=None,
                cat_features=None,
                random_seed=2023
    ):
    scores = []
    models = []

    kf = KFold(n_splits=3, shuffle=True, random_state=random_seed)

    print(f"========= TRAINING {algorithm.__name__} =========")

    for num_fold, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_eval = # YOUR CODE
        y_train, y_eval = # YOUR CODE

        if init_params is not None:
            model = algorithm(**init_params)
        else:
            model = algorithm()

        if algorithm.__name__ == 'CatBoostRegressor':
            # Используйте соответствующий класс
            train_dataset = # YOUR CODE
            eval_dataset = # YOUR CODE

            model.fit(train_dataset,
                      eval_set=eval_dataset,
                      verbose=0,
                      early_stopping_rounds=early_stopping_rounds)

        elif algorithm.__name__ == 'LGBMRegressor':
            # Используйте соответствующий класс
            train_dataset = # YOUR CODE
            eval_dataset = # YOUR CODE

            model = lgb.train(params=init_params,
                              train_set=train_dataset,
                              valid_sets=(eval_dataset),
                              categorical_feature=cat_features,
                              verbose_eval=False)

        elif algorithm.__name__ == 'XGBRegressor':
            # Используйте соответствующий класс
            train_dataset = # YOUR CODE
            eval_dataset = # YOUR CODE

            model = xgb.train(params=init_params,
                              dtrain=train_dataset,
                              evals=[(train_dataset, 'dtrain'), (eval_dataset, 'dtest')],
                              verbose_eval=False,
                              early_stopping_rounds=early_stopping_rounds)

            X_eval = eval_dataset

        # Сделайте предсказание на X_eval и посчитайте RMSE
        y_pred = # YOUR CODE
        score = # YOUR CODE

        models.append(model)
        scores.append(score)

        print(f'FOLD {num_fold}: SCORE {score}')

    mean_kfold_score = np.mean(scores, dtype="float16") -  np.std(scores, dtype="float16")
    print("\nMEAN RMSE SCORE", mean_kfold_score)

    # Выберите модель с наименьшим значением скора
    best_model = # YOUR CODE

    return mean_kfold_score, best_model

In [ ]:
def tuning_hyperparams(algorithm,
                       X,
                       y,
                       init_params,
                       fit_params,
                       grid_params,
                       n_iter,
                       cv=3,
                       random_state=2023,
    ):

    estimator = algorithm(**init_params)

    # Можно использоавть GridSearchCV
    model = RandomizedSearchCV(estimator=estimator,
                               param_distributions=grid_params,
                               n_iter=n_iter,
                               cv=cv,
                               scoring='neg_root_mean_squared_error',
                               n_jobs=-1,
                               verbose=0,
                               random_state=random_state
    )

    model.fit(X, y, **fit_params)

    return model.best_params_ | init_params

# 4️⃣ **Группируем признаки, отбираем категориальные, выделяем датасет для обучения.**

In [ ]:
cat_features = [...]
targets = [...]
features2drop = [...]

filtered_features = [...]
num_features = [...]


print("cat_features", cat_features)
print("num_features", num_features)
print("targets", targets)

In [ ]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["target_reg"]

# 5️⃣ **CatBoostRegressor.**



## **Обучение модели.**

In [ ]:
cb_init_params = {
    'loss_function': ...,
    'eval_metric': ...,
    'thread_count': -1,
    'task_type': 'CPU',
    'random_seed': RANDOM_STATE
}

cb_score, cb_model = train_model(
    algorithm=CatBoostRegressor,
    X=X, y=y,
    init_params=cb_init_params,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [ ]:
cb_test_pred = ... # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': cb_test_pred}).to_csv('cb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'CatBoostRegressor',
    'tuning': False,
    'kfold_score': cb_score,
    'leaderboard_score': ...,
    'model': cb_model
})

## **Подбор гиперпараметров и обучение модели с новыми параметрами.**

In [ ]:
cb_fit_params = {
    'cat_features': cat_features,
    'verbose': 0,
    'early_stopping_rounds': ...
}


# Параметры, которые будем перебирать
cb_grid_params = {
    'depth': ...,
    'learning_rate': ...,
    # YOUR PARAMS
}


catboost_params_after_tuning = tuning_hyperparams(algorithm=CatBoostRegressor,
                                                  X=X, y=y,
                                                  init_params=cb_init_params,
                                                  fit_params=cb_fit_params,
                                                  grid_params=cb_grid_params,
                                                  n_iter=...,
                                                  cv=...,
                                                  random_state=RANDOM_STATE
)

catboost_params_after_tuning

In [ ]:
cb_tuning_score, cb_tuning_model = train_model(algorithm=CatBoostRegressor,
                                               X=X, y=y,
                                               early_stopping_rounds=...,
                                               init_params=catboost_params_after_tuning,
                                               cat_features=cat_features,
                                               random_seed=RANDOM_STATE)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [ ]:
tuning_cb_test_pred = # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': tuning_cb_test_pred}).to_csv('tuning_cb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'CatBoostRegressor',
    'tuning': True,
    'mean_kfold_score': cb_tuning_score,
    'leaderboard_score': ...,
    'model': cb_tuning_model
})

# 6️⃣ **LightGBMRegressor (goss).**

## **Подготовка категориальных признаков.**

[Ссылка](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.2%20-%20LightGBM.ipynb), если забыли, как готовить категориальные признаки

In [ ]:
X_lgb = X.copy()

le = LabelEncoder()

### YOUR CODE

## **Обучение модели.**

In [ ]:
lgb_init_params = {
    'boosting_type': ...,
    'n_jobs': -1,
    'metric': ...,
    'objective': ...,
    'random_state': RANDOM_STATE,
    'verbosity': -1,
    'device': 'cpu',
}


lgb_score, lgb_model = train_model(
    algorithm=LGBMRegressor,
    X=X_lgb, y=y,
    init_params=lgb_init_params,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [ ]:
lgb_test_pred = # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': lgb_test_pred}).to_csv('lgb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'LGBMRegressor (goss)',
    'tuning': False,
    'mean_kfold_score': lgb_score,
    'leaderboard_score': ...,
    'model': lgb_model
})

## **Подбор гиперпараметров и обучение модели с новыми параметрами**

In [ ]:
lgb_fit_params = {
    'eval_metric': 'rmse',
    'categorical_feature': cat_features
}


lgb_grid_params = {
    'max_depth': ...,
    'min_data_in_leaf': ...,
    # YOUR PARAMS
}


lgb_params_after_tuning = tuning_hyperparams(algorithm=LGBMRegressor,
                                             X=X_lgb, y=y,
                                             init_params=lgb_init_params,
                                             fit_params=lgb_fit_params,
                                             grid_params=lgb_grid_params,
                                             n_iter=...,
                                             cv=...,
                                             random_state=RANDOM_STATE
)

lgb_params_after_tuning

In [ ]:
lgb_tuning_score, lgb_tuning_model = train_model(
    algorithm=LGBMRegressor,
    X=X_lgb, y=y,
    init_params=lgb_params_after_tuning,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [ ]:
results.append({
    'model_name': 'LGBMRegressor (goss)',
    'tuning': True,
    'mean_kfold_score': lgb_tuning_score,
    'leaderboard_score': ...,
    'model': lgb_tuning_model
})

# 7️⃣ **XGBoostRegressor (dart).**

## **Подготовка категориальных признаков.**

[Ссылка](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.3%20-%20XGBoost.ipynb), если забыли, как готовить категориальные признаки

In [ ]:
X_xgb = X.copy()

# YOUR CODE

## **Обучение модели.**

In [ ]:
xgb_init_params = {
    'enable_categorical': True,
    'booster': ...,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbosity': 0,

    # параметры, которые обязательно объявить, чтобы модель работала в режиме dart
    ...: ...,
    ...: ...,
}


xgb_score, xgb_model = train_model(
    algorithm=XGBRegressor,
    X=X_xgb, y=y,
    init_params=xgb_init_params,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [ ]:
xgb_test_pred = ... # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': xgb_test_pred}).to_csv('xgb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'XGBRegressor (dart)',
    'tuning': False,
    'mean_kfold_score': xgb_score,
    'leaderboard_score': ...,
    'model': xgb_model
})

## **Подбор гиперпараметров и обучение модели с новыми параметрами**

In [ ]:
xgb_grid_params = {
    'max_depth': ...,
    'max_leaves': ...,
    # YOUR PARAMS
}


xgb_fit_params = {
    'verbose': False
}


xgb_params_after_tuning = tuning_hyperparams(algorithm=XGBRegressor,
                                             X=X_xgb, y=y,
                                             init_params=xgb_init_params,
                                             fit_params=xgb_fit_params,
                                             grid_params=xgb_grid_params,
                                             n_iter=...,
                                             cv=...,
                                             random_state=RANDOM_STATE
)

xgb_params_after_tuning

In [ ]:
xgb_tuning_score, xgb_tuning_model = train_model(
    algorithm=XGBRegressor,
    X=X_xgb, y=y,
    init_params=xgb_params_after_tuning,
    early_stopping_rounds=...,
    cat_features=cat_features,
    random_seed=RANDOM_STATE
)

In [ ]:
tuning_xgb_test_pred = ... # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': tuning_xgb_test_pred}).to_csv('tuning_xgb_pred.csv', index=False)

In [ ]:
results.append({
    'model_name': 'XGBRegressor (dart)',
    'tuning': True,
    'mean_kfold_score': xgb_tuning_score,
    'leaderboard_score': ...,
    'model': xgb_tuning_model
})

# 8️⃣ **Финальное предсказание и сохранение лучших моделей**

In [ ]:
best_cb_model = # YOUR CODE
best_cb_model.save_model('best_cb_model.cbm')

best_lgb_model = # YOUR CODE
best_lgb_model.save_model('best_lgb_model.mod')


best_xgb_model = # YOUR CODE
best_xgb_model.save_model('best_xgb_model.json')

In [ ]:
final_cb_pred = # YOUR CODE
final_lgb_pred = # YOUR CODE
final_xgb_pred = # YOUR CODE

final_pred = # YOUR CODE

pd.DataFrame({'car_id': test['car_id'], 'target_reg': final_pred}).to_csv('final_submission.csv', index=False)

# 9️⃣ **Выводы.**


In [ ]:
results = pd.DataFrame(results)
results

Примеры вопросов, на которые можно ответить при формулировании вывода:

- Какая модель показала лучшее качество на валидации/лидерборде?
- Помог ли тюнинг гиперпараметров?
- Помог ли Feature Selection?
- Помог ли Object Selection?
- Что поняли благодаря построенным графикам?
- Улучшилось ли качество на лидерборде после усреднения прогнозов моделей?
- ...

